In [4]:
# Imports
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import Sequence

In [5]:
# Class For Data Generation
class DataGenerator(Sequence):
    def __init__(self, path='./neuralzome_crate_local/2024-01-31-09-51-48/rgb/', batch_size=16, shuffle=True):
        self.path = path
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        # Filter yaml with no annotations
        self.images = []
        for file in sorted(os.listdir(self.path)):
            if file.endswith('.jpg'):
                yaml_path = os.path.join(self.path, file.replace('.jpg', '.yaml'))
                if os.path.exists(yaml_path):
                    with open(yaml_path, 'r') as f:
                        data = yaml.safe_load(f)
                        if data.get('crates'):
                            self.images.append(file)

        self.on_epoch_end()

    # Length of the dataset
    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))
    
    # Shuffling the dataset
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.images)

    # Get a batch of data
    def __getitem__(self, index):
        batch_files = self.images[index * self.batch_size:(index + 1) * self.batch_size] # 
        images = []
        keypoints = []

        for file in batch_files:
            img_path = os.path.join(self.path, file)
            yaml_path = os.path.join(self.path, file.replace('.jpg', '.yaml'))

            # Load image
            img = Image.open(img_path).convert('RGB')
            img = np.array(img).astype('float32') / 255.0

            # Load keypoints
            with open(yaml_path, 'r') as f:
                crate = yaml.safe_load(f)['crates'][0]
            kp = np.array([
                crate['x0'], crate['y0'],
                crate['x1'], crate['y1'],
                crate['x2'], crate['y2'],
                crate['x3'], crate['y3']
            ], dtype='float32')

            images.append(img)
            keypoints.append(kp)

        return np.stack(images), np.stack(keypoints)

In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
import tensorflow as tf

# Custom loss
def keypoint_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Load MobileNetV2 without top, with input shape (480, 640, 3)
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(480, 640, 3))

# Freeze layers if desired
for layer in base_model.layers:
    layer.trainable = False

# Build Sequential model and add base model as a layer
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(8))

# Compile
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=keypoint_loss,
    metrics=['mae']
)

dataset = DataGenerator()

# Train the model
model.fit(dataset, epochs=40)

/var/folders/vs/h43bw_9d56z8mpjr0qs4v7nm0000gn/T/ipykernel_1152/918806619.py:11: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(480, 640, 3))


Epoch 1/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 973ms/step - loss: 95029.7812 - mae: 270.9493
Epoch 2/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 858ms/step - loss: 45657.0938 - mae: 172.4327
Epoch 3/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 19099.4844 - mae: 111.2376
Epoch 4/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 12585.7803 - mae: 86.7466
Epoch 5/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 784ms/step - loss: 13747.6133 - mae: 93.6433
Epoch 6/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 951ms/step - loss: 10881.3809 - mae: 79.3706
Epoch 7/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 9950.5713 - mae: 79.2719
Epoch 8/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 820ms/step - loss: 8445.9834 - mae: 73.6041
Epoch 9/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 991ms/step - loss: 9772.1152 - mae: 77.3959
Epoch 10/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 7874.7373 - mae: 69.5034 
Epoch 11/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 8368.1104 - mae: 72.5382 
Epoch 12/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 917ms/step - loss: 8241.7754 - mae: 71.5793
Epoc

# Left this here because resize works much better out of pocket